In [9]:
%run static_data.ipynb
%run solution.ipynb

In [10]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import os
import time

data = None
gg_map = None
gg_data = None

In [11]:
class GGMap:
    def __init__(self, sub_data):
        self.gg_map = sub_data
        self.mapping()
        
    def mapping(self):
        self.layers = self.gg_map['figure']._trait_values['_map'].layers[:]
        self.makers = self.layers[0]
        self.direction_layers = self.layers[1:]
        self.current_data_bounds = self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'][:]
        
    def retain_direction_layer_with_index(self, idx):
        self.direction_layers = self.layers[1:]
        self.direction_layers = self.direction_layers[idx]
        self.gg_map['figure']._trait_values['_map'].layers = [self.makers] + [self.direction_layers]
#         self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'] = self.current_data_bounds
        print(self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'])

In [12]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

def handle_execute_btn(obj):
    global tour_btn, data, gg_data
    with output:
        exe_btn.disabled = True
        data = execute(w_select.value[:-4], metric='duration', layout='100%')
        if data is not None:
            map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
            grid[:-2, 2:-1] = map_cont
            tour_btn.options = data['route']
            gg_data = GGMap(data)
        time.sleep(1)
    exe_btn.disabled = False
        
def handle_tour_select(change):
    global gg_data
    with output:
        desc_map = dict(zip(data['route'], data['route_description']))
        dir_btn.value = desc_map[tour_btn.value]
        if gg_data:
            gg_data.retain_direction_layer_with_index(int(tour_btn.value[:tour_btn.value.rfind(' ->')]))
            map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
            grid[:-2, 2:-1] = map_cont

In [13]:
w_select = widgets.Select(
    options=os.listdir('../data/tw_data/'),
    value=None,
    disabled=False, layout=widgets.Layout(height='100%', width='100%')
)

exe_btn = create_expanded_button('Execute', 'success')
map_btn = widgets.Button(description='', button_style='', layout=widgets.Layout(height='auto', width='auto'))
map_cont = widgets.Box([map_btn], layout=widgets.Layout(height='100%', width='100%', align_items='stretch'))
tour_btn = widgets.Select(options=[], value=None, description='', disabled=False, layout=widgets.Layout(height='100%', width='100%'))
dir_btn = widgets.HTML(value='Empty', placeholder='Empty', disabled=True, layout=widgets.Layout(height='auto', width='auto'))
dir_cont = widgets.HBox([dir_btn], layout=widgets.Layout(height='100%', overflow_y='auto'))

In [14]:
grid = widgets.GridspecLayout(10, 16, height='800px')
grid[:-2, 2:-1] = map_cont
grid[1:, :2] = w_select
grid[0, :2] = exe_btn
grid[:, -1:] = tour_btn
grid[-2:, 2:-1] = dir_cont

output = widgets.Output()
exe_btn.on_click(handle_execute_btn)
tour_btn.observe(handle_tour_select, names='value')

In [15]:
display(grid, output)

GridspecLayout(children=(Box(children=(Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle())…

Output()